In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn import cross_validation
import xgboost as xgb

def ToWeight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w

def rmspe(yhat, y):
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
    return rmspe

def rmspe_xg(yhat, y):
    # y = y.values
    y = y.get_label()
    y = np.exp(y) - 1
    yhat = np.exp(yhat) - 1
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w * (y - yhat)**2))
    return "rmspe", rmspe

In [3]:
train = pd.read_csv('train_nnew.csv', low_memory=False)
test = pd.read_csv('test_nnew.csv', low_memory=False)
features = [u'Open', u'Promo', u'SchoolHoliday', u'StateHoliday_0',
       u'StateHoliday_a', u'DayOfWeek_1', u'DayOfWeek_2', u'DayOfWeek_3',
       u'DayOfWeek_4', u'DayOfWeek_5', u'DayOfWeek_6', u'DayOfWeek_7',
       u'CompetitionDistance', u'Promo2', 'year', 'Mean_Sales', 'month', 'day',
       u'StoreType_a', u'StoreType_b', u'StoreType_c', u'StoreType_d',
       u'Assortment_a', u'Assortment_b', u'Assortment_c', u'CompetitionOpen']

In [4]:
train['year'] = train.Date.apply(lambda x: x.split('-')[0])
train['year'] = train['year'].astype(float)
train['month'] = train.Date.apply(lambda x: x.split('-')[1])
train['month'] = train['month'].astype(float)
train['day'] = train.Date.apply(lambda x: x.split('-')[2])
train['day'] = train['day'].astype(float)

In [5]:
test['year'] = test.Date.apply(lambda x: x.split('-')[0])
test['year'] = test['year'].astype(float)
test['month'] = test.Date.apply(lambda x: x.split('-')[1])
test['month'] = test['month'].astype(float)
test['day'] = test.Date.apply(lambda x: x.split('-')[2])
test['day'] = test['day'].astype(float)

In [6]:
train.columns = [u'Store', u'DayOfWeek', u'Date', u'Sales', u'Customers', u'Open',
       u'Promo', u'StateHoliday', u'SchoolHoliday', u'StateHoliday0',
       u'StateHolidaya', u'DayOfWeek1', u'DayOfWeek2', u'DayOfWeek3',
       u'DayOfWeek4', u'DayOfWeek5', u'DayOfWeek6', u'DayOfWeek7',
       u'CompetitionDistance', u'Promo2', u'CompetitionOpenSince',
       u'StoreTypea', u'StoreTypeb', u'StoreTypec', u'StoreTyped',
       u'Assortmenta', u'Assortmentb', u'Assortmentc', u'CompetitionOpen',
       u'MeanSales', u'year', u'month', u'day']

In [7]:
test.columns = [u'Id', u'Store', u'DayOfWeek', u'Date', u'Open', u'Promo',
       u'StateHoliday', u'SchoolHoliday', u'StateHolidaya', u'StateHoliday0',
       u'DayOfWeek1', u'DayOfWeek2', u'DayOfWeek3', u'DayOfWeek4',
       u'DayOfWeek5', u'DayOfWeek6', u'DayOfWeek7', u'CompetitionDistance',
       u'Promo2', u'CompetitionOpenSince', u'StoreTypea', u'StoreTypeb',
       u'StoreTypec', u'StoreTyped', u'Assortmenta', u'Assortmentb',
       u'Assortmentc', u'CompetitionOpen', u'MeanSales', u'year', u'month',
       u'day']

In [8]:
features = [u'Open', u'Promo', u'StateHoliday0',
       u'StateHolidaya', u'DayOfWeek1', u'DayOfWeek2', u'DayOfWeek3',
       u'DayOfWeek4', u'DayOfWeek5', u'DayOfWeek6', u'DayOfWeek7',
       u'CompetitionDistance', u'Promo2', 'year', 'MeanSales', 'month', 'day',
       u'StoreTypea', u'StoreTypeb', u'StoreTypec', u'StoreTyped',
       u'Assortmenta', u'Assortmentb', u'Assortmentc', u'CompetitionOpen']

In [9]:
ts_features = [u'Open', u'Promo', u'StateHoliday0',
       u'StateHolidaya', u'DayOfWeek1', u'DayOfWeek2', u'DayOfWeek3',
       u'DayOfWeek4', u'DayOfWeek5', u'DayOfWeek6', u'DayOfWeek7',
       u'CompetitionDistance', u'Promo2', 'year', 'MeanSales', 'month', 'day', u'CompetitionOpen']

In [10]:
train['Date'] = train['Date'].astype('datetime64')
test['Date'] = test['Date'].astype('datetime64')
train['CompetitionOpenSince'] = train['CompetitionOpenSince'].astype('datetime64')
test['CompetitionOpenSince'] = test['CompetitionOpenSince'].astype('datetime64')

In [11]:
train_x = train[train['Store'] == 1]

In [12]:
train_x = train_x.sort(['Date'])

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [13]:
train_x.index = np.arange(len(train_x.index))

In [28]:
stores = np.unique(test.Store)

In [14]:
stores = [1]

In [15]:
store = 1

In [16]:
cols_nosh = [               u'Store',            u'DayOfWeek',              u'Sales',            u'Open',
                      u'Promo',         
              u'SchoolHoliday',        u'StateHoliday0',
              u'StateHolidaya',           u'DayOfWeek1',
                 u'DayOfWeek2',           u'DayOfWeek3',
                 u'DayOfWeek4',           u'DayOfWeek5',
                 u'DayOfWeek6',           u'DayOfWeek7',
        u'CompetitionDistance',               u'Promo2',           u'StoreTypea',
                 u'StoreTypeb',           u'StoreTypec',
                 u'StoreTyped',          u'Assortmenta',
                u'Assortmentb',          u'Assortmentc',
            u'CompetitionOpen',            u'MeanSales',
                       u'year',                u'month',
                        u'day']

In [17]:
allfs = features + [i+'mean' for i in ts_features] + [i+'std' for i in ts_features] + [i+'dta' for i in ts_features]

In [64]:
train_7.loc[6,[i+'dta' for i in cols_nosh]] = (train_x.loc[max(6-1, 0),cols_nosh] - train_x.loc[max(6-6, 0),cols_nosh]).values

In [18]:
train_x = train[train['Store'] == store]
train_x = train_x.sort(['Date'])
train_x.index = np.arange(len(train_x.index))
test_x = test[test['Store'] == store]
test_x = test_x.sort(['Date'])
test_x.index = np.arange(len(test_x.index))
train_7 = train_x.copy()
train_18 = train_x.copy()
train_30 = train_x.copy()
test_7 = test_x.copy()
test_18 = test_x.copy()
test_30 = test_x.copy()
for f in [i+'mean' for i in ts_features] + [i+'std' for i in ts_features] + [i+'dta' for i in ts_features]:
    train_7.loc[:,f] = range(len(train_7))
    test_7.loc[:,f] = range(len(test_7))
for step in xrange(6, len(train_7)):
    s = max(step-6,0)
    st = max(step-1, 0)
    train_7.loc[step,[i+'mean' for i in ts_features]] = (train_x.loc[s:st,ts_features].mean(axis=0)).values
    train_7.loc[step,[i+'std' for i in ts_features]] = (train_x.loc[s:st,ts_features].std(axis=0)).values
    train_7.loc[step,[i+'dta' for i in ts_features]] = (train_x.loc[st,ts_features] - \
                                                        train_x.loc[s,ts_features]).values
    #train_7.loc[step-6,'Sales'] = train_x.loc[step+1,'Sales']
    #train_7.loc[step-6,'Open'] = train_x.loc[step+1,'Open']
train_7 = train_7.loc[6:,:]
for step in xrange(len(test_7)):
    s = max(step-6,0)
    st = max(step-1, 0)
    test_7.loc[step,[i+'mean' for i in ts_features]] = (test_x.loc[s:st,ts_features].mean(axis=0)).values
    test_7.loc[step,[i+'std' for i in ts_features]] = (test_x.loc[s:st,ts_features].std(axis=0)).values
    test_7.loc[step,[i+'dta' for i in ts_features]] = (test_x.loc[st,ts_features] - \
                                                       train_x.loc[s,ts_features]).values

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [19]:
test_7 = test_7.fillna(0)

In [24]:
from hyperopt import hp, fmin, tpe

In [29]:
space = (hp.quniform('eta', 0.01, 1),
         hp.quniform('subsample', 0.01, 1),
         hp.quniform('colsample_bytree', 0.01, 1),
         hp.quniform('lambda', 0, 1000),
         hp.quniform('alpha', 0, 1),
         hp.randint('max_depth', 1, 30))

In [14]:
X_train, X_test = cross_validation.train_test_split(train, test_size=0.05, random_state = 1)
dtrain = xgb.DMatrix(X_train[X_train['Open'] > 0][features], np.log(X_train[X_train['Open'] > 0]["Sales"] + 1))
dvalid = xgb.DMatrix(X_test[X_test['Open'] > 0][features], np.log(X_test[X_test['Open'] > 0]["Sales"] + 1))

def calc(params):

    print params
    
    num_trees = 5000
    
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, early_stopping_rounds=50, 
                    feval=rmspe_xg, verbose_eval=False)

    #print("Validating")
    train_probs = gbm.predict(dvalid)
    indices = train_probs < 0
    train_probs[indices] = 0
    error = rmspe(np.exp(train_probs) - 1, X_test[X_test['Open'] > 0]['Sales'].values)
    #print('error', error)
    return error

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss
from sklearn import preprocessing

import numpy as np
import pandas as pd

from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

def score(params):
    print "Training with params : "
    print params
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_test, label=y_test)
    # watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    model = xgb.train(params, dtrain, num_round)
    predictions = model.predict(dvalid).reshape((X_test.shape[0], 9))
    score = log_loss(y_test, predictions)
    print "\tScore {0}\n\n".format(score)
    return {'loss': score, 'status': STATUS_OK}


def optimize():
    space = {
             'eta' : hp.quniform('eta', 0.025, 0.5, 0.025),
             'max_depth' : hp.quniform('max_depth', 1, 20, 1),
             'subsample' : hp.quniform('subsample', 0.2, 1, 0.05),
             'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
             'colsample_bytree' : hp.quniform('colsample_bytree', 0.2, 1, 0.05),
             'alpha' : hp.quniform('alpha', 0, 1, 0.05),
             'lambda' : hp.quniform('lambda', 0, 1000, 1),
             'objective': 'reg:linear',
             'silent' : 1
             }

    best = fmin(calc, space, algo=tpe.suggest, max_evals=2500)

    print best


#Trials object where the history of search will be stored
#trials = Trials()

optimize()

In [ ]:
params = {"objective": "reg:linear", 
          "eta": 0.5, 
          "max_depth": 8, 
          "subsample": 0.7, 
          "colsample_bytree": 0.7, 
          "silent": 1, 
          #"lambda" : 100, 
          #"alpha" : 1
         } 
num_trees = 100

In [40]:
X_train, X_test = cross_validation.train_test_split(train_7, test_size=0.05, random_state = 1)
dtrain = xgb.DMatrix(X_train[X_train['Open'] > 0][allfs], np.log(X_train[X_train['Open'] > 0]["Sales"] + 1))
dvalid = xgb.DMatrix(X_test[X_test['Open'] > 0][allfs], np.log(X_test[X_test['Open'] > 0]["Sales"] + 1))
dtest = xgb.DMatrix(test_7[allfs])
watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, early_stopping_rounds=50, 
                feval=rmspe_xg, verbose_eval=True)

print("Validating")
train_probs = gbm.predict(dvalid)
indices = train_probs < 0
train_probs[indices] = 0
error = rmspe(np.exp(train_probs) - 1, X_test[X_test['Open'] > 0]['Sales'].values)
print('error', error)

#test_probs_18 = gbm.predict(dtest)
#indices = test_probs_18 < 0
#test_probs_18[indices] = 0

Will train until train error hasn't decreased in 50 rounds.
[0]	eval-rmspe:0.981450	train-rmspe:0.981210
[1]	eval-rmspe:0.863841	train-rmspe:0.861923
[2]	eval-rmspe:0.635097	train-rmspe:0.631027
[3]	eval-rmspe:0.412703	train-rmspe:0.400980
[4]	eval-rmspe:0.268685	train-rmspe:0.241932
[5]	eval-rmspe:0.183928	train-rmspe:0.153977
[6]	eval-rmspe:0.158886	train-rmspe:0.110336
[7]	eval-rmspe:0.138767	train-rmspe:0.088372
[8]	eval-rmspe:0.134259	train-rmspe:0.074500
[9]	eval-rmspe:0.128839	train-rmspe:0.068683
[10]	eval-rmspe:0.128966	train-rmspe:0.062903
[11]	eval-rmspe:0.133170	train-rmspe:0.055506
[12]	eval-rmspe:0.133585	train-rmspe:0.051931
[13]	eval-rmspe:0.134014	train-rmspe:0.048617
[14]	eval-rmspe:0.132504	train-rmspe:0.046849
[15]	eval-rmspe:0.135449	train-rmspe:0.043702
[16]	eval-rmspe:0.136765	train-rmspe:0.041930
[17]	eval-rmspe:0.140413	train-rmspe:0.040519
[18]	eval-rmspe:0.135486	train-rmspe:0.037454
[19]	eval-rmspe:0.133426	train-rmspe:0.034702
[20]	eval-rmspe:0.133223	train

Validating
('error', 0.13416380876664089)


[88]	eval-rmspe:0.133993	train-rmspe:0.002180
[89]	eval-rmspe:0.133997	train-rmspe:0.002180
[90]	eval-rmspe:0.133994	train-rmspe:0.002180
[91]	eval-rmspe:0.134006	train-rmspe:0.002173
[92]	eval-rmspe:0.134009	train-rmspe:0.002154
[93]	eval-rmspe:0.134011	train-rmspe:0.002154
[94]	eval-rmspe:0.134015	train-rmspe:0.002155
[95]	eval-rmspe:0.134027	train-rmspe:0.002139
[96]	eval-rmspe:0.134014	train-rmspe:0.002135
[97]	eval-rmspe:0.134022	train-rmspe:0.002136
[98]	eval-rmspe:0.134150	train-rmspe:0.002129
[99]	eval-rmspe:0.134164	train-rmspe:0.002129


In [136]:
ans = test[['Id']]

In [120]:
from sklearn.ensemble import RandomForestRegressor

In [29]:
train_a7 = pd.DataFrame(columns=allfs)
train_a18 = pd.DataFrame(columns=allfs)
train_a30 = pd.DataFrame(columns=allfs)
test_a7 = pd.DataFrame(columns=allfs + ['Id'])
test_a18 = pd.DataFrame(columns=allfs + ['Id'])
test_a30 = pd.DataFrame(columns=allfs + ['Id'])

In [30]:
for store in stores:
    print 'store num', store
    train_x = train[train['Store'] == store]
    train_x = train_x.sort(['Date'])
    train_x.index = np.arange(len(train_x.index))
    test_x = test[test['Store'] == store]
    test_x = test_x.sort(['Date'])
    test_x.index = np.arange(len(test_x.index))
    train_7 = train_x.copy()
    train_18 = train_x.copy()
    train_30 = train_x.copy()
    test_7 = test_x.copy()
    test_18 = test_x.copy()
    test_30 = test_x.copy()
    for f in [i+'mean' for i in ts_features] + [i+'std' for i in ts_features] + [i+'dta' for i in ts_features]:
        train_7.loc[:,f] = range(len(train_7))
        test_7.loc[:,f] = range(len(test_7))
        train_18.loc[:,f] = range(len(train_18))
        test_18.loc[:,f] = range(len(test_18))
        train_30.loc[:,f] = range(len(train_30))
        test_30.loc[:,f] = range(len(test_30))
    for step in xrange(6, len(train_7)):
        s = max(step-6,0)
        st = max(step-1, 0)
        train_7.loc[step,[i+'mean' for i in ts_features]] = (train_x.loc[s:st,ts_features].mean(axis=0)).values
        train_7.loc[step,[i+'std' for i in ts_features]] = (train_x.loc[s:st,ts_features].std(axis=0)).values
        train_7.loc[step,[i+'dta' for i in ts_features]] = (train_x.loc[st,ts_features] - \
                                                            train_x.loc[s,ts_features]).values
    train_7 = train_7.loc[6:,:]
    for step in xrange(len(test_7)):
        s = max(step-6,0)
        st = max(step-1, 0)
        test_7.loc[step,[i+'mean' for i in ts_features]] = (test_x.loc[s:st,ts_features].mean(axis=0)).values
        test_7.loc[step,[i+'std' for i in ts_features]] = (test_x.loc[s:st,ts_features].std(axis=0)).values
        test_7.loc[step,[i+'dta' for i in ts_features]] = (test_x.loc[st,ts_features] - \
                                                           train_x.loc[s,ts_features]).values
    test_7 = test_7.fillna(0)
    
    for step in xrange(17, len(train_18)):
        s = max(step-17,0)
        st = max(step-1, 0)
        train_18.loc[step,[i+'mean' for i in ts_features]] = (train_x.loc[s:st,ts_features].mean(axis=0)).values
        train_18.loc[step,[i+'std' for i in ts_features]] = (train_x.loc[s:st,ts_features].std(axis=0)).values
        train_18.loc[step,[i+'dta' for i in ts_features]] = (train_x.loc[st,ts_features] - \
                                                             train_x.loc[s,ts_features]).values
    train_18 = train_18.loc[17:,:]
    for step in xrange(len(test_18)):
        s = max(step-17,0)
        st = max(step-1, 0)
        test_18.loc[step,[i+'mean' for i in ts_features]] = (test_x.loc[s:st,ts_features].mean(axis=0)).values
        test_18.loc[step,[i+'std' for i in ts_features]] = (test_x.loc[s:st,ts_features].std(axis=0)).values
        test_18.loc[step,[i+'dta' for i in ts_features]] = (test_x.loc[st,ts_features] - \
                                                            train_x.loc[s,ts_features]).values
    test_18 = test_18.fillna(0)
    
    for step in xrange(29, len(train_30)):
        s = max(step-29,0)
        st = max(step-1, 0)
        train_30.loc[step,[i+'mean' for i in ts_features]] = (train_x.loc[s:st,ts_features].mean(axis=0)).values
        train_30.loc[step,[i+'std' for i in ts_features]] = (train_x.loc[s:st,ts_features].std(axis=0)).values
        train_30.loc[step,[i+'dta' for i in ts_features]] = (train_x.loc[st,ts_features] - \
                                                             train_x.loc[s,ts_features]).values
    train_30 = train_30.loc[29:,:]
    for step in xrange(len(test_30)):
        s = max(step-29,0)
        st = max(step-1, 0)
        test_30.loc[step,[i+'mean' for i in ts_features]] = (test_x.loc[s:st,ts_features].mean(axis=0)).values
        test_30.loc[step,[i+'std' for i in ts_features]] = (test_x.loc[s:st,ts_features].std(axis=0)).values
        test_30.loc[step,[i+'dta' for i in ts_features]] = (test_x.loc[st,ts_features] - \
                                                            train_x.loc[s,ts_features]).values
    test_30 = test_30.fillna(0)
    
    train_a7 = pd.concat([train_a7, train_7])
    train_a18 = pd.concat([train_a18, train_18])
    train_a30 = pd.concat([train_a30, train_30])
    test_a7 = pd.concat([test_a7, test_7])
    test_a18 = pd.concat([test_a18, test_18])
    test_a30 = pd.concat([test_a30, test_30])
    
    #ans.loc[test_7.Id-1, 'Sales_7'] = np.exp(test_probs_7) - 1
    #ans.loc[test_18.Id-1, 'Sales_18'] = np.exp(test_probs_18) - 1
    #ans.loc[test_30.Id-1, 'Sales_30'] = np.exp(test_probs_30) - 1
    #ans['Sales'] = ans.loc[:, ['Sales_7','Sales_18','Sales_30']].mean(axis=1)

store num 1
store num 3
store num 7
store num 8
store num 9
store num 10
store num 11
store num 12
store num 13
store num 14
store num 15
store num 16


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


KeyboardInterrupt: 

In [139]:
print 'OLOLO'

OLOLO


In [140]:
ans[['Id', 'Sales']].to_csv("rf_ts_ind.csv", index=False)

In [141]:
ans

,Id,Sales_7,Sales_18,Sales_30,Sales
0,1,4251.382482,4168.469634,4103.432784,4174.428300
1,2,7642.082397,7560.451238,7827.801700,7676.778445
2,3,7735.843515,8044.769232,8473.068434,8084.560393
3,4,7137.878685,6824.343136,6937.614738,6966.612186
4,5,6141.323294,6192.697763,5655.505637,5996.508898
5,6,5440.689182,5710.045655,5792.587536,5647.774124
6,7,7228.827077,7413.990474,7950.286864,7531.034805
7,8,8191.153401,7715.232008,8211.151985,8039.179131
8,9,5633.846755,5173.021793,5842.806794,5549.891781
9,10,5434.562313,5799.087453,5969.275602,5734.308456


In [ ]:
train_a7.to_csv('train_a7.csv', index = False)
train_a18.to_csv('train_a18.csv', index = False)
train_a30.to_csv('train_a30.csv', index = False)
test_a7.to_csv('test_a7.csv', index = False)
test_a18.to_csv('test_a18.csv', index = False)
test_a30.to_csv('test_a30.csv', index = False)

In [55]:
len(train_x)

942

In [117]:
train_x.loc[0:0]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StateHoliday_0,...,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,CompetitionOpen,Mean_Sales,year,month,day
0,1,2,2013-01-01,0,0,0,0,a,1,0,...,1,0,1,0,0,1,3835.285714,2013,1,1


In [36]:
train_x.loc[0:6]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StateHoliday_0,...,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,CompetitionOpen,Mean_Sales,year,month,day
0,1,2,2013-01-01,0,0,0,0,a,1,0,...,1,0,1,0,0,1,3835.285714,2013,1,1
1,1,3,2013-01-02,5530,668,1,0,0,1,1,...,1,0,1,0,0,1,3746.746032,2013,1,2
2,1,4,2013-01-03,4327,578,1,0,0,1,1,...,1,0,1,0,0,1,3419.396825,2013,1,3
3,1,5,2013-01-04,4486,619,1,0,0,1,1,...,1,0,1,0,0,1,4206.063492,2013,1,4
4,1,6,2013-01-05,4997,635,1,0,0,1,1,...,1,0,1,0,0,1,4942.970149,2013,1,5
5,1,7,2013-01-06,0,0,0,0,0,1,1,...,1,0,1,0,0,1,0.000000,2013,1,6
6,1,1,2013-01-07,7176,785,1,1,0,1,1,...,1,0,1,0,0,1,5852.708333,2013,1,7


In [45]:
train_z = train_x.loc[0:6]

In [46]:
train_z.loc[0,:] = train_x.loc[0:6].mean(axis=0)

In [48]:
train_z.loc[1,:] = train_x.loc[1:7].mean(axis=0)

In [50]:
train_x.loc[1:7].mean(axis=0)

Store                     1.000000
DayOfWeek                 4.000000
Sales                  4585.142857
Customers               562.714286
Open                      0.857143
Promo                     0.285714
SchoolHoliday             1.000000
StateHoliday_0            1.000000
StateHoliday_a            0.000000
DayOfWeek_1               0.142857
DayOfWeek_2               0.142857
DayOfWeek_3               0.142857
DayOfWeek_4               0.142857
DayOfWeek_5               0.142857
DayOfWeek_6               0.142857
DayOfWeek_7               0.142857
CompetitionDistance    1270.000000
Promo2                    0.000000
StoreType_a               0.000000
StoreType_b               0.000000
StoreType_c               1.000000
StoreType_d               0.000000
Assortment_a              1.000000
Assortment_b              0.000000
Assortment_c              0.000000
CompetitionOpen           1.000000
Mean_Sales             3933.211722
year                   2013.000000
month               

In [49]:
train_z

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StateHoliday_0,...,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,CompetitionOpen,Mean_Sales,year,month,day
0,1,4,NaT,3788.000000,469.285714,0.714286,0.142857,NaN,1,0.857143,...,1,0,1,0,0,1,3697.517640,2013,1,4.428571
1,1,4,NaT,4585.142857,562.714286,0.857143,0.285714,NaN,1,1.000000,...,1,0,1,0,0,1,3933.211722,2013,1,5.000000
2,1,4,2013-01-03,4327.000000,578.000000,1.000000,0.000000,0,1,1.000000,...,1,0,1,0,0,1,3419.396825,2013,1,3.000000
3,1,5,2013-01-04,4486.000000,619.000000,1.000000,0.000000,0,1,1.000000,...,1,0,1,0,0,1,4206.063492,2013,1,4.000000
4,1,6,2013-01-05,4997.000000,635.000000,1.000000,0.000000,0,1,1.000000,...,1,0,1,0,0,1,4942.970149,2013,1,5.000000
5,1,7,2013-01-06,0.000000,0.000000,0.000000,0.000000,0,1,1.000000,...,1,0,1,0,0,1,0.000000,2013,1,6.000000
6,1,1,2013-01-07,7176.000000,785.000000,1.000000,1.000000,0,1,1.000000,...,1,0,1,0,0,1,5852.708333,2013,1,7.000000
